In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from CMA_obj import CMA_opt
from PEPG_obj import PEPG_opt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from SPSA_obj import SPSA_opt
from ADAM_opt import AdamOptimizer
from PSO_obj import PSO_opt
from scipy.interpolate import interp1d
from numpy import asarray
from numpy import savetxt
from NN_utils import *
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import pandas as pd

#   Online Training of Neural Networks IRIS, Wine

- The NN class helper functions and training loop functions are defined in NN_utils, 

### Loading datasets
X is the input, Y the output

In [ ]:
#Iris dataset
iris_df = pd.read_csv("data\\IRIS\\iris.csv")
# convert the last column 


iris_raw = iris_df.values

for i in range(len(iris_raw)):
    if iris_raw[i,-1] == 'Iris-setosa':
        iris_raw[i,-1] = 0
    elif iris_raw[i,-1] == 'Iris-versicolor':
        iris_raw[i,-1] = 1
    else:
        iris_raw[i,-1] = 2
        
iris_raw = iris_raw.astype(np.float32)
#iris raw needs to be shuffled randomly because the data is ordered by class
np.random.shuffle(iris_raw)

# Convert to PyTorch tensors
X = torch.from_numpy(iris_raw[:, :-1])
Y = torch.from_numpy(iris_raw[:, -1]).unsqueeze(1)

# Create a single dataset
full_dataset = Custom_dataset(X, Y)

# Split into train and test sets, first set the size of the split
train_size = int(0.75 * len(full_dataset))
test_size = len(full_dataset) - train_size
# split into train and test sets using pytorch randomsplit
Iris_train, Iris_test = torch.utils.data.random_split(full_dataset, [train_size, test_size])

Iris_train_loader = torch.utils.data.DataLoader(dataset=Iris_train, batch_size=train_size, shuffle=True)
Iris_test_loader = torch.utils.data.DataLoader(dataset=Iris_test, batch_size=test_size, shuffle=False)

In [12]:
# Wine dataset
wine_df = pd.read_csv("data\\WINE\\winequality-red.csv")

wine_raw = wine_df.values.astype(np.float32)

# Convert to PyTorch tensors
X = torch.from_numpy(wine_raw[:, :-1])
Y = torch.from_numpy(wine_raw[:, -1]).unsqueeze(1)

# Create a single dataset
full_dataset = Custom_dataset(X, Y)

# Split into train and test sets, first set the size of the split
train_size = int(0.75 * len(full_dataset))
test_size = len(full_dataset) - train_size
# split into train and test sets using pytorch randomsplit

Wine_train, Wine_test = torch.utils.data.random_split(full_dataset, [train_size, test_size])


Wine_train_loader = torch.utils.data.DataLoader(dataset=Wine_train, batch_size=100, shuffle=True)
Wine_test_loader = torch.utils.data.DataLoader(dataset=Wine_test, batch_size=100, shuffle=False)

In [15]:
#We Now create an instance of the NN class and move if to the GPU if available
n_neurons = 10
NN_Wine = Neural_Net(input_size=11, hidden_size=n_neurons, n_classes=1)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NN_Wine.parameters(), lr=0.001)

#training the full NN
n_epochs = 1000
test_acc = train_pytorch_NN(NN_Wine, n_epochs, Wine_train_loader, Wine_test_loader, loss, optimizer)

Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=11, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc)), y=test_acc, mode='lines', name='Full NN'))
fig.update_layout(template='plotly_white', width=400, height=400,margin=dict(l=20, r=20, t=20, b=20))


In [ ]:
# Training loop PEPG for MNIST: 


#NN_MNIST.reset_weights()
#NN_MNIST.NN_stack[0].requires_grad = True
n_epochs =10
NN_MNIST = Tiny_convnet()
N_dim = NN_MNIST.count_parameters()
pop_size = 200

#specify we don't need the computation graph to keep track of the gradients, we will use pepg to update the weights
with torch.no_grad():
    for param in NN_MNIST.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters


init_pos = NN_MNIST.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

PEPG_optimizer = PEPG_opt(N_dim, pop_size, learning_rate=0.01, starting_mu=init_pos ,starting_sigma=0.1)

PEPG_optimizer.sigma_decay = 0.9999
PEPG_optimizer.sigma_alpha=0.2
PEPG_optimizer.sigma_limit=0.02
PEPG_optimizer.elite_ratio=0.1
PEPG_optimizer.weight_decay=0.005

test_acc_PEPG,best_reward_PEPG = train_online_pop_NN(NN_MNIST, n_epochs, train_loader_MNIST, test_loader_MNIST, loss, PEPG_optimizer)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_PEPG)), y=test_acc_PEPG, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [ ]:
#save pepg data
savetxt('data\\Results\\NN_training\\online_training\\PEPG_test_acc.csv', test_acc_PEPG, delimiter=',')

### We use CMA to train the FFNN
- This doesn't work at all this simple architecture has too many parameters ... so CMA is painfully slow.

In [ ]:
#Using CMA-ES for training the NN
n_epochs =10
NN_MNIST = Tiny_convnet()
N_dim = NN_MNIST.count_parameters()
pop_size = 200
#specify we don't need the computation graph to keep track of the gradients, we will use CMAES to update the weights
with torch.no_grad():
    for param in NN_MNIST.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters

init_pos = NN_MNIST.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

CMA_optimizer = CMA_opt(N_dim, pop_size, select_pop=int(pop_size/2), sigma_init=0.1, mean_init=init_pos)
CMA_optimizer.eigen_update_frequency = 10

test_acc_CMA,best_reward_CMA = train_online_pop_NN(NN_MNIST, n_epochs, train_loader_MNIST, test_loader_MNIST, loss, CMA_optimizer)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_CMA)), y=test_acc_CMA, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [ ]:
#use SPSA to optimize The Neural network
n_epochs =10
NN_MNIST = Tiny_convnet()
N_dim = NN_MNIST.count_parameters()
#specify we don't need the computation graph to keep track of the gradients, we will use SPSA to update the weights
with torch.no_grad():
    for param in NN_MNIST.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters

init_pos = NN_MNIST.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

SPSA_optimizer = SPSA_opt(init_pos,alpha=1e-3,epsilon=1e-7)
Adam = AdamOptimizer(init_pos, lr=1e-3, beta1=0.9, beta2=0.99, epsilon=1e-8)

test_acc_SPSA, best_reward_SPSA = train_online_SPSA_NN(NN_MNIST, n_epochs, train_loader_MNIST, test_loader_MNIST, loss, SPSA_optimizer,Adam)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_SPSA)), y=test_acc_SPSA, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [ ]:
#Using PSO for training the NN
n_epochs =1
NN_MNIST = Tiny_convnet()
N_dim = NN_MNIST.count_parameters()
pop_size = 100
#specify we don't need the computation graph to keep track of the gradients, we will use CMAES to update the weights
with torch.no_grad():
    for param in NN_MNIST.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters

init_pos = NN_MNIST.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

#params dictionary
upper_bound = 0.5
lower_bound = -0.5

params = {'c_1': 2.5, 
          'c_2': 0.85,
          'w': 0.7,
          'Vmax': 0.15*(upper_bound-lower_bound),
          'upper_bound': upper_bound,
          'lower_bound': lower_bound,
          'pop_size' :pop_size,
          }

init_pos = (upper_bound - lower_bound) * np.random.rand(N_dim, pop_size) + lower_bound
V_init = 0.1 * np.random.rand(N_dim, pop_size)
PSO_optimizer = PSO_opt(X_init = init_pos,V_init = V_init,params=params)

test_acc_PSO,best_reward_PSO = train_online_pop_NN(NN_MNIST, n_epochs, train_loader_MNIST, test_loader_MNIST, loss, PSO_optimizer)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_PSO)), y=test_acc_PSO, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [ ]:
model = NN_MNIST

# Initialize a list to store the figures
figs = []

# Iterate through each model parameter
for name, param in model.named_parameters():
    if 'weight' in name:  # Filter out only weight parameters
        # Flatten the weights
        weights = param.detach().cpu().numpy().flatten()
        
        # Create a histogram for the weights
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=weights, name=name))
        
        # Update layout to add titles and improve readability
        fig.update_layout(
            title=f'Histogram of Weights for Layer: {name}',
            xaxis_title='Weight values',
            yaxis_title='Frequency',
            bargap=0.2
        )
        
        # Append the figure to the list
        figs.append(fig)

# Show all histograms
for fig in figs:
    fig.show()